# Ршение задач PostgresSQL

## Посчитайте, сколько компаний закрылось.

In [29]:
"""
SELECT COUNT(status)
FROM company
GROUP BY status 
HAVING status LIKE 'closed';
"""

"\nSELECT COUNT(status)\nFROM company\nGROUP BY status \nHAVING status LIKE 'closed';\n"

## Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [30]:
"""
SELECT funding_total
FROM company
WHERE country_code='USA'
  AND category_code LIKE 'news' 
ORDER BY funding_total DESC
"""

"\nSELECT funding_total\nFROM company\nWHERE country_code='USA'\n  AND category_code LIKE 'news' \nORDER BY funding_total DESC\n"

## Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [18]:
"""
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
  AND (CAST(acquired_at as date) BETWEEN '2011-01-01' AND '2013-12-31')
"""

"\nSELECT SUM(price_amount)\nFROM acquisition\nWHERE term_code = 'cash'\n  AND (CAST(acquired_at as date) BETWEEN '2011-01-01' AND '2013-12-31')\n"

## Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на 'Silver'.

In [19]:
"""
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';
"""

"\nSELECT first_name,\n       last_name,\n       twitter_username\nFROM people\nWHERE twitter_username LIKE 'Silver%';\n"

## Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.

In [20]:
"""
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
  AND last_name LIKE 'K%'
"""

"\nSELECT *\nFROM people\nWHERE twitter_username LIKE '%money%'\n  AND last_name LIKE 'K%'\n"

## Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [21]:
"""
SELECT SUM(funding_total),
       country_code
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC
"""

'\nSELECT SUM(funding_total),\n       country_code\nFROM company\nGROUP BY country_code\nORDER BY SUM(funding_total) DESC\n'

##  Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [22]:
"""
SELECT CAST(funded_at AS DATE),
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY CAST(funded_at AS DATE)
HAVING  MIN(raised_amount) > 0 
 AND MIN(raised_amount) !=  MAX(raised_amount)
"""

'\nSELECT CAST(funded_at AS DATE),\n       MIN(raised_amount),\n       MAX(raised_amount)\nFROM funding_round\nGROUP BY CAST(funded_at AS DATE)\nHAVING  MIN(raised_amount) > 0 \n AND MIN(raised_amount) !=  MAX(raised_amount)\n'

## Создайте поле с категориями:

- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
- Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.

Отобразите все поля таблицы fund и новое поле с категориями.

In [23]:
"""
SELECT *,
       CASE 
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies < 100 AND  invested_companies >= 20 THEN 'middle_activity'
           WHEN invested_companies < 20 THEN 'low_activity'
       END
FROM fund;
"""

"\nSELECT *,\n       CASE \n           WHEN invested_companies >= 100 THEN 'high_activity'\n           WHEN invested_companies < 100 AND  invested_companies >= 20 THEN 'middle_activity'\n           WHEN invested_companies < 20 THEN 'low_activity'\n       END\nFROM fund;\n"

## Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [24]:
"""
WITH
res as
    (SELECT *,
           CASE
               WHEN invested_companies>=100 THEN 'high_activity'
               WHEN invested_companies>=20 THEN 'middle_activity'
               ELSE 'low_activity'
           END AS activity
    FROM fund)
SELECT activity,
       ROUND(AVG(investment_rounds))
FROM res
GROUP BY activity
ORDER BY AVG(investment_rounds)
"""

"\nWITH\nres as\n    (SELECT *,\n           CASE\n               WHEN invested_companies>=100 THEN 'high_activity'\n               WHEN invested_companies>=20 THEN 'middle_activity'\n               ELSE 'low_activity'\n           END AS activity\n    FROM fund)\nSELECT activity,\n       ROUND(AVG(investment_rounds))\nFROM res\nGROUP BY activity\nORDER BY AVG(investment_rounds)\n"

## Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю.

Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [25]:
"""
SELECT MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies),
       country_code
FROM fund
WHERE (EXTRACT(YEAR FROM (CAST(founded_at AS DATE))) BETWEEN 2010 AND 2012)

GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY  AVG(invested_companies) DESC, country_code
LIMIT 10
"""

'\nSELECT MIN(invested_companies),\n       MAX(invested_companies),\n       AVG(invested_companies),\n       country_code\nFROM fund\nWHERE (EXTRACT(YEAR FROM (CAST(founded_at AS DATE))) BETWEEN 2010 AND 2012)\n\nGROUP BY country_code\nHAVING MIN(invested_companies) > 0\nORDER BY  AVG(invested_companies) DESC, country_code\nLIMIT 10\n'

## Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [26]:
"""
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id
"""

'\nSELECT p.first_name,\n       p.last_name,\n       e.instituition\nFROM people AS p\nLEFT JOIN education AS e ON p.id = e.person_id\n'

## Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [27]:
"""
WITH
res1 AS 
    (SELECT e.person_id,
            p.company_id,
            e.instituition 
    FROM education AS e
    INNER JOIN people AS p ON e.person_id = p.id
    WHERE instituition  IS NOT NULL
      AND e.person_id IN (SELECT id
                          FROM people
                          )),

res2 AS 
    (SELECT com.id, com.name
    FROM company AS com)

SELECT res2.name, COUNT(DISTINCT res1.instituition)
FROM res1
JOIN res2 ON res1.company_id = res2.id
GROUP BY res2.name
ORDER BY COUNT(DISTINCT res1.instituition) DESC
LIMIT 5
"""

'\nWITH\nres1 AS \n    (SELECT e.person_id,\n            p.company_id,\n            e.instituition \n    FROM education AS e\n    INNER JOIN people AS p ON e.person_id = p.id\n    WHERE instituition  IS NOT NULL\n      AND e.person_id IN (SELECT id\n                          FROM people\n                          )),\n\nres2 AS \n    (SELECT com.id, com.name\n    FROM company AS com)\n\nSELECT res2.name, COUNT(DISTINCT res1.instituition)\nFROM res1\nJOIN res2 ON res1.company_id = res2.id\nGROUP BY res2.name\nORDER BY COUNT(DISTINCT res1.instituition) DESC\nLIMIT 5\n'

## Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [28]:
"""
SELECT  DISTINCT name
FROM company
WHERE status = 'closed'
  AND id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1
               AND is_last_round = 1)
"""

"\nSELECT  DISTINCT name\nFROM company\nWHERE status = 'closed'\n  AND id IN (SELECT company_id\n             FROM funding_round\n             WHERE is_first_round = 1\n               AND is_last_round = 1)\n"

## Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
"""
WITH
res1 AS
(SELECT  DISTINCT name, id
FROM company
WHERE status = 'closed'
  AND id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1
               AND is_last_round = 1))

SELECT p.id
FROM people AS p
INNER JOIN res1 ON res1.id = p.company_id

"""

## Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [32]:
"""
WITH
res1 AS
(SELECT  DISTINCT name, id
FROM company
WHERE status = 'closed'
  AND id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1
               AND is_last_round = 1))

SELECT DISTINCT p.id, e.instituition
FROM people AS p
INNER JOIN res1 ON res1.id = p.company_id
INNER JOIN education AS e ON p.id = e.person_id
"""

"\nWITH\nres1 AS\n(SELECT  DISTINCT name, id\nFROM company\nWHERE status = 'closed'\n  AND id IN (SELECT company_id\n             FROM funding_round\n             WHERE is_first_round = 1\n               AND is_last_round = 1))\n\nSELECT DISTINCT p.id, e.instituition\nFROM people AS p\nINNER JOIN res1 ON res1.id = p.company_id\nINNER JOIN education AS e ON p.id = e.person_id\n"

## Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [33]:
"""
WITH
res1 AS
(SELECT  DISTINCT name, id
FROM company
WHERE status = 'closed'
  AND id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1
               AND is_last_round = 1))

SELECT DISTINCT COUNT(e.instituition) OVER (PARTITION BY p.id),
       p.id
FROM people AS p
INNER JOIN res1 ON res1.id = p.company_id
INNER JOIN education AS e ON p.id = e.person_id
"""

"\nWITH\nres1 AS\n(SELECT  DISTINCT name, id\nFROM company\nWHERE status = 'closed'\n  AND id IN (SELECT company_id\n             FROM funding_round\n             WHERE is_first_round = 1\n               AND is_last_round = 1))\n\nSELECT DISTINCT COUNT(e.instituition) OVER (PARTITION BY p.id),\n       p.id\nFROM people AS p\nINNER JOIN res1 ON res1.id = p.company_id\nINNER JOIN education AS e ON p.id = e.person_id\n"

## Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [34]:
"""
WITH
res1 AS
(SELECT  DISTINCT name, id
FROM company
WHERE status = 'closed'
  AND id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1
               AND is_last_round = 1))

SELECT AVG(res2.cnt) FROM
    (SELECT DISTINCT COUNT(e.instituition) OVER (PARTITION BY p.id) AS cnt,
           p.id
    FROM people AS p
    INNER JOIN res1 ON res1.id = p.company_id
    INNER JOIN education AS e ON p.id = e.person_id) AS res2
"""

"\nWITH\nres1 AS\n(SELECT  DISTINCT name, id\nFROM company\nWHERE status = 'closed'\n  AND id IN (SELECT company_id\n             FROM funding_round\n             WHERE is_first_round = 1\n               AND is_last_round = 1))\n\nSELECT AVG(res2.cnt) FROM\n    (SELECT DISTINCT COUNT(e.instituition) OVER (PARTITION BY p.id) AS cnt,\n           p.id\n    FROM people AS p\n    INNER JOIN res1 ON res1.id = p.company_id\n    INNER JOIN education AS e ON p.id = e.person_id) AS res2\n"

## Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.

*(сервис, запрещённый на территории РФ)

In [35]:
"""
WITH
res1 AS
(SELECT  DISTINCT name, id
FROM company
WHERE name = 'Facebook')

SELECT AVG(res2.cnt) FROM
    (SELECT DISTINCT COUNT(e.instituition) OVER (PARTITION BY p.id) AS cnt,
           p.id
    FROM people AS p
    INNER JOIN res1 ON res1.id = p.company_id
    INNER JOIN education AS e ON p.id = e.person_id) AS res2
"""

"\nWITH\nres1 AS\n(SELECT  DISTINCT name, id\nFROM company\nWHERE name = 'Facebook')\n\nSELECT AVG(res2.cnt) FROM\n    (SELECT DISTINCT COUNT(e.instituition) OVER (PARTITION BY p.id) AS cnt,\n           p.id\n    FROM people AS p\n    INNER JOIN res1 ON res1.id = p.company_id\n    INNER JOIN education AS e ON p.id = e.person_id) AS res2\n"

## Составьте таблицу из полей:

- name_of_fund — название фонда;

- name_of_company — название компании;

- amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [36]:
"""
WITH 
res1 AS 
    (SELECT id,
           name AS company_name,
           milestones
    FROM company
    WHERE milestones >=6
      AND id IN (SELECT company_id
                 FROM funding_round
                 WHERE EXTRACT(YEAR FROM (CAST(funded_at AS DATE))) BETWEEN 2012 AND 2013))
                 
SELECT DISTINCT f.name,
       res1.company_name,
       fr.raised_amount
FROM investment AS i
JOIN res1 ON i.company_id=res1.id
JOIN fund AS f ON i.fund_id=f.id
JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2012 AND 2013
   AND res1.milestones > 6;
"""

'\nWITH \nres1 AS \n    (SELECT id,\n           name AS company_name,\n           milestones\n    FROM company\n    WHERE milestones >=6\n      AND id IN (SELECT company_id\n                 FROM funding_round\n                 WHERE EXTRACT(YEAR FROM (CAST(funded_at AS DATE))) BETWEEN 2012 AND 2013))\n                 \nSELECT DISTINCT f.name,\n       res1.company_name,\n       fr.raised_amount\nFROM investment AS i\nJOIN res1 ON i.company_id=res1.id\nJOIN fund AS f ON i.fund_id=f.id\nJOIN funding_round AS fr ON i.funding_round_id = fr.id\nWHERE EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2012 AND 2013\n   AND res1.milestones > 6;\n'

## Выгрузите таблицу, в которой будут такие поля:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [37]:
"""
SELECT c.name AS acquiring_company, cc.name AS acquired_company,
       a.price_amount, cc.funding_total,
       ROUND(a.price_amount / cc.funding_total)
FROM acquisition AS a
LEFT JOIN company AS c ON a.acquiring_company_id=c.id
LEFT JOIN company AS cc ON a.acquired_company_id=cc.id
WHERE a.price_amount !=0 
  AND cc.funding_total > 0

ORDER by a.price_amount DESC, cc.name
LIMIT 10

"""

'\nSELECT c.name AS acquiring_company, cc.name AS acquired_company,\n       a.price_amount, cc.funding_total,\n       ROUND(a.price_amount / cc.funding_total)\nFROM acquisition AS a\nLEFT JOIN company AS c ON a.acquiring_company_id=c.id\nLEFT JOIN company AS cc ON a.acquired_company_id=cc.id\nWHERE a.price_amount !=0 \n  AND cc.funding_total > 0\n\nORDER by a.price_amount DESC, cc.name\nLIMIT 10\n\n'

## Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [38]:
"""
SELECT c.name,
       temp_t.month_n
FROM company as c
    JOIN
    (SELECT company_id, EXTRACT(MONTH FROM (CAST(funded_at AS DATE))) AS month_n
    FROM funding_round
    WHERE (EXTRACT(YEAR FROM (CAST(funded_at AS DATE))) BETWEEN 2010 AND 2013)
      AND raised_amount > 0) AS temp_t
     ON temp_t.company_id = c.id
     WHERE category_code = 'social'
"""

"\nSELECT c.name,\n       temp_t.month_n\nFROM company as c\n    JOIN\n    (SELECT company_id, EXTRACT(MONTH FROM (CAST(funded_at AS DATE))) AS month_n\n    FROM funding_round\n    WHERE (EXTRACT(YEAR FROM (CAST(funded_at AS DATE))) BETWEEN 2010 AND 2013)\n      AND raised_amount > 0) AS temp_t\n     ON temp_t.company_id = c.id\n     WHERE category_code = 'social'\n"

## Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:

- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
"""
WITH
-- выбираем месяц инвестиционных раундов в 2010-2013 гг
tab1 AS (SELECT EXTRACT(MONTH FROM funded_at) AS month,
                id AS funding_round_id
                FROM funding_round
         WHERE EXTRACT(YEAR FROM funded_at) BETWEEN 2010 AND 2013
         ),

-- считаем кол-во купленных и общую сумму по сделкам за 2010-2013 гг в разрезе месяца
tab2 AS (SELECT EXTRACT(MONTH FROM acquired_at) AS month,
                COUNT(acquired_company_id) AS count_acquired,
                SUM(price_amount) AS total_amount
         FROM acquisition
         WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
         GROUP BY EXTRACT(MONTH FROM acquired_at)
        ),

-- ищем фонды из США
tab3 AS (SELECT i.funding_round_id,
                f.name
        FROM investment AS i
        JOIN fund AS f ON f.id = i.fund_id
        WHERE fund_id IN (SELECT id
                          FROM fund
                          WHERE country_code LIKE 'USA')
        ),

tab4 AS (SELECT month,
                COUNT(DISTINCT name) AS count_USA
         FROM tab1 
         LEFT JOIN tab3 ON tab1.funding_round_id = tab3.funding_round_id
         GROUP BY month)
         
SELECT tab4.month,
       tab4.count_USA,
       tab2.count_acquired,
       tab2.total_amount
FROM tab4 
LEFT JOIN tab2 ON tab4.month = tab2.month;
"""

## Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
"""
WITH
y_2011 AS (SELECT country_code,
                  AVG(funding_total) AS avg_2011
           FROM  company
           WHERE EXTRACT(YEAR FROM founded_at) = 2011
             --AND funding_total > 0
           GROUP BY country_code),
          
y_2012 AS (SELECT country_code,
                  AVG(funding_total) AS avg_2012
           FROM  company
           WHERE EXTRACT(YEAR FROM founded_at) = 2012
             --AND funding_total > 0
           GROUP BY country_code),

y_2013 AS (SELECT country_code,
                  AVG(funding_total) AS avg_2013
           FROM  company
           WHERE EXTRACT(YEAR FROM founded_at) = 2013
             --AND funding_total > 0
           GROUP BY country_code)

SELECT y_2011.country_code, 
       y_2011.avg_2011,
       y_2012.avg_2012,
       y_2013.avg_2013
FROM y_2011
JOIN y_2012 ON y_2011.country_code = y_2012.country_code
JOIN y_2013 ON y_2011.country_code = y_2013.country_code
ORDER BY y_2011.avg_2011 DESC
"""